# Exploratory data analysis (EDA) : trip statistics

# Purpose
Some statistics have been generated for each trip. This will be explored in this notebook, to find some possible patterns in the data.

# Methodology
* Load the statistics.
* Descriptive statistics.
* Make some nice seaborn plots
* Generate heat map.

# Setup

In [ ]:
# %load imports.py
# %load ../imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
width=20
height=3
plt.rcParams["figure.figsize"] = (width,height)
sns.set(rc={'figure.figsize':(width,height)})

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score

import scipy.integrate
import seaborn as sns

import pyarrow as pa
import pyarrow.parquet as pq

import dask.dataframe
import statsmodels.api as sm

from d2e2f.visualization import visualize
import scipy.integrate

from d2e2f.pipelines.trip_statistics import clean_statistics

In [ ]:
loaded = catalog.load('trip_statistics_clean')
df_stat = loaded['tycho-brahe-wind-2021-04-10--2021-06-30.csv']()
df_stat.head()

In [ ]:
df_stat.describe()

In [ ]:
df_stat.shape

In [ ]:
df_stat.info()

In [ ]:
df_stat.describe()

In [ ]:
df_stat['trip_direction'].unique()

Some data is missing, so that the trip_direction always has the sequence:0,1,0,... cannot be taken for granted:

## Time and direction matter

In [ ]:
fig = px.scatter(df_stat, x='start_time',y='P', color='trip_direction', width=1500, height=600, 
                color_discrete_sequence=['red','green'], hover_data=['trip_no','trip_time'])
fig.show()

In [ ]:
fig = px.scatter(df_stat, x='start_time',y='sog', color='trip_direction', width=1500, height=600, 
                color_discrete_sequence=['red','green'], hover_data=['trip_no','trip_time'])
fig.show()

In [ ]:
sns.displot(df_stat, x='P', hue='trip_direction', binwidth=25, aspect=3)

In [ ]:
sns.displot(df_stat, x='P', hue='trip_direction', kind="kde", bw_adjust=2, aspect=3)

## Heat maps

In [ ]:
df = df_stat.groupby(by='trip_direction').get_group('Helsingør-Helsingborg').copy()
df.drop(columns=['end_time','trip_direction'], inplace=True)

In [ ]:
df.describe()

In [ ]:
selection = [f'P{i}' for i in range(1,5)]
df_selection = df[selection]
corr = df_selection.corr().abs()
ax = sns.heatmap(corr, vmin=0, vmax=1, yticklabels=corr.index, cmap='Blues')
fig = ax.get_figure()
fig.set_size_inches(9,9)

In [ ]:
g = sns.PairGrid(df_selection)
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)

<a id='P_corr'></a>
* P1 and P2 are highly correlated
* P3 and P4 are highly correlated

In [ ]:
selection = [f'sin_pm{i}' for i in range(1,5)]
df_selection = df[selection]
corr = df_selection.corr().abs()

ax = sns.heatmap(corr, vmin=0, vmax=1, yticklabels=corr.index, cmap='Blues')
fig = ax.get_figure()
fig.set_size_inches(9,9)

* sin_pm1 and sin_pm2 are highly correlated
* sin_pm3 and sin_pm4 are highly correlated

In [ ]:
g = sns.PairGrid(df_selection)
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)

In [ ]:
selection = [f'cos_pm{i}' for i in range(1,5)]
df_selection = df[selection]
corr = df_selection.corr().abs()

ax = sns.heatmap(corr, vmin=0, vmax=1, yticklabels=corr.index, cmap='Blues')
fig = ax.get_figure()
fig.set_size_inches(9,9)

* cos_pm1 and cos_pm2 are highly correlated
* cos_pm3 and cos_pm4 are also correlated but not as much!

<a id='fwd_cos'></a>

In [ ]:
g = sns.PairGrid(df_selection)
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)

* There seams to be something strange with the cos_pm3 and cos_pm4 data

In [ ]:
df2 = df.copy()

mask = ((df2['P1']==0) |
        (df2['P2']==0) |
        (df2['P3']==0) |
        (df2['P4']==0) )

df2 = df2.loc[~mask].copy()

P_fwd_ratio = (df2['P1']/df2['P2'])
P_aft_ratio = (df2['P3']/df2['P4'])
max_ratio_diff = 0.1
mask = (P_fwd_ratio.between((1-max_ratio_diff), (1+max_ratio_diff)) &
        P_aft_ratio.between((1-max_ratio_diff), (1+max_ratio_diff)))
df2 = df2.loc[mask].copy() 

P_fwd_ratio = (df2['cos_pm1']/df2['cos_pm2'])
P_aft_ratio = (df2['cos_pm3']/df2['cos_pm4'])
max_ratio_diff = 0.3
mask = (P_fwd_ratio.between((1-max_ratio_diff), (1+max_ratio_diff)) &
        P_aft_ratio.between((1-max_ratio_diff), (1+max_ratio_diff)))
df2 = df2.loc[mask].copy() 


df2['P_fwd'] = df2['P1'] + df2['P2']
df2['P_aft'] = df2['P3'] + df2['P4']
df2['sin_pm_fwd'] = (df2['sin_pm1'] + df2['sin_pm2'])/2
df2['cos_pm_fwd'] = (df2['cos_pm1'] + df2['cos_pm2'])/2
df2['sin_pm_aft'] = (df2['sin_pm3'] + df2['sin_pm4'])/2
df2['cos_pm_aft'] = (df2['cos_pm3'] + df2['cos_pm4'])/2

removes = []
removes+=([f'P{i}' for i in range(1,5)])
removes+=([f'sin_pm{i}' for i in range(1,5)])
removes+=([f'cos_pm{i}' for i in range(1,5)])

df2.drop(columns=removes, inplace=True)

In [ ]:
df2.head()

In [ ]:
corr = df2.corr().abs()

ax = sns.heatmap(corr, vmin=0, vmax=1, yticklabels=corr.index, cmap='Blues')
fig = ax.get_figure()
fig.set_size_inches(9,9)

In [ ]:
df_selection = df2.select_dtypes(exclude='object').copy()

selection = list(df_selection.columns)
selection.remove('P_fwd')
selection.remove('P_aft')
selection.remove('sin_pm_aft')
selection.remove('cos_pm_aft')
selection.remove('sin_pm_fwd')
selection.remove('cos_pm_fwd')
selection.remove('reversing')
selection.remove('E')
selection.remove('E1')
selection.remove('E2')
selection.remove('E3')
selection.remove('E4')

#selection.remove('start_time')
selection.remove('trip_time')  # Very correlated with sog
#selection.remove('aw')
selection.remove('Effekt Propulsion Total (kW)')

df_selection = df_selection[selection].copy()

corr = df_selection.corr().abs()

grid = sns.heatmap(corr, vmin=0, vmax=1, cmap='Blues', annot=True)
grid.set_xticklabels(grid.get_xticklabels(), rotation = 90)
fig = grid.get_figure()

fig.set_size_inches(10,10)

In [ ]:
df_selection.columns

In [ ]:
mask = corr.loc['P'] > 0.25
df_selection2 = df_selection[df_selection.columns[mask]]

x_vars=list(df_selection2.columns)
x_vars.remove('P')
g = sns.PairGrid(df_selection2, y_vars=["P"], x_vars=x_vars, height=4)
g.map(sns.scatterplot, color=".3")


In [ ]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

def fit_predict(y,X):
    
    model = sm.OLS(y,X)
    results = model.fit()
    display(results.summary())
    
    sog = np.linspace(X.index.min(), X.index.max(), 100)
    y_pred = results.predict(X)
    prstd, iv_l, iv_u = wls_prediction_std(results, exog=X, alpha=0.05)
    
    data = pd.DataFrame(index=X.index)
    data['sog'] = X.index
    data['P'] = y 
    
    grid = sns.relplot(x="sog", y="P", data=data);
    
    grid.ax.plot(X.index, y_pred, 'r')
    grid.ax.plot(X.index, iv_l, 'k--')
    grid.ax.plot(X.index, iv_u, 'k--')
    
    display(grid)
    
    return results

In [ ]:
data = df_selection[['P','sog']].copy()
data.sort_values(by='sog', inplace=True)
data.set_index('sog', inplace=True)

y = data.pop('P')
X = data
X['sog**3'] = X.index**3

fit_predict(y=y, X=X)

<a id='sog'></a>
...so it seems that that a cubic resistance model can describe 99.4% of the variance in the trip mean power.

In [ ]:
sns.jointplot(data=df_selection, x="sog", y="P", kind="kde", fill=True, height=10)

In [ ]:
def fit_predict2(y,X):
    
    model = sm.OLS(y,X)
    results = model.fit()
    display(results.summary())
    
    sog = np.linspace(X.index.min(), X.index.max(), 100)
    y_pred = results.predict(X)
    prstd, iv_l, iv_u = wls_prediction_std(results, exog=X, alpha=0.05)
    
    #data = pd.DataFrame(index=X.index)
    #data['sog'] = X.index
    #data['P'] = y 
    
    #grid = sns.relplot(x="sog", y="P", data=data);
    
    #grid.ax.plot(X.index, y_pred, 'r')
    
    #grid.ax.plot(X.index, iv_l, 'k--')
    #grid.ax.plot(X.index, iv_u, 'k--')
    
    fig,ax=plt.subplots()
    ax.plot(y, y_pred, '.')
    ax.set_aspect('equal', 'box')
    ax.plot([np.min(y), np.max(y)], [np.min(y), np.max(y)], 'r-')
    ax.set_ylabel('Y_pred')
    ax.set_xlabel('Y')
    
    
    return results

In [ ]:
data = df_selection2.copy()
data.sort_values(by='sog', inplace=True)
data.set_index('sog', inplace=True)

y = data.pop('P')
X = data
X['sog**3'] = X.index**3

fit_predict2(y=y, X=X)

# Following trips
How does the correlation, for a certain trip direction, look between a trip and following trips. 

In [ ]:
df = df_stat.groupby('trip_direction').get_group('Helsingborg-Helsingør')
df1 = df.iloc[0:-1]
df2 = df.iloc[1:]

df_energy_following = pd.DataFrame(index=df1.index)
df_energy_following['mean(power) [kW] at trips'] = df1['P'].values.copy()
df_energy_following['mean(power) [kW] at the following trips'] = df2['P'].values.copy()

grid = sns.displot(
    data=df_energy_following,
    x='mean(power) [kW] at trips', y='mean(power) [kW] at the following trips',
    kind="kde", height=10, fill=True,
)


ax=grid.ax

ax.plot(df1['P'], df2['P'], 'r.', alpha=0.20, zorder=10)
ax.set_xlabel('power_em_thruster_total at trips')
ax.set_ylabel('power_em_thruster_total at the followin trips');



In [ ]:
df = df_stat.groupby('trip_direction').get_group('Helsingør-Helsingborg')
df1 = df.iloc[0:-1]
df2 = df.iloc[1:]

df_energy_following = pd.DataFrame(index=df1.index)
df_energy_following['mean(power) [kW] at trips'] = df1['P'].values.copy()
df_energy_following['mean(power) [kW] at the following trips'] = df2['P'].values.copy()

grid = sns.displot(
    data=df_energy_following,
    x='mean(power) [kW] at trips', y='mean(power) [kW] at the following trips',
    kind="kde", height=10, fill=True,
)


ax=grid.ax

ax.plot(df1['P'], df2['P'], 'r.', alpha=0.20, zorder=10)
ax.set_xlabel('power_em_thruster_total at trips')
ax.set_ylabel('power_em_thruster_total at the followin trips');

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(df_selection['P']);

In [ ]:
y = df_selection['P'].values

def calcualte_autocorrelation(y,lag = 1):
    
    if lag==0:
        return 1.0
    else:
        y_mean = np.mean(y)
        return np.sum((y[0:-lag]-y_mean)*(y[lag:]-y_mean)) / np.sum((y-y_mean)**2)

autocorrelations = []
for lag in np.arange(0,35):
    autocorrelation = calcualte_autocorrelation(y=y, lag=lag)
    autocorrelations.append(autocorrelation)
    
fig,ax=plt.subplots()
ax.plot(autocorrelations)

In [ ]:
np.sum((y-y_mean)**2)

In [ ]:
np.var(y)

In [ ]:
np.std(y)

In [ ]:
np.sqrt(np.sum((y-y_mean)**2)/len(y))

In [ ]:
np.std(y)**2

In [ ]:
np.var(y)